In [1]:
import transformers

#Set to avoid warning messages.
transformers.logging.set_verbosity_error()

In [6]:
verbose_text ="""
On October 28, 2003, 19-year-old Harvard University sophomore Mark Zuckerberg is dumped by his girlfriend,
 Erica Albright. Returning to his dorm, Zuckerberg writes an insulting post about Albright on his LiveJournal blog.
 He creates a campus website called Facemash by downloading photos of female students from house face books,
 then allowing site visitors to rate their attractiveness. After traffic to the site crashes parts of Harvard's
  computer network, Zuckerberg is given six months of academic probation. However, Facemash's popularity attracts
  the attention of twins Cameron and Tyler Winklevoss and their business partner Divya Narendra. The trio invites
   Zuckerberg to work on Harvard Connection, a social network exclusive to Harvard students and aimed at dating.
    Zuckerberg approaches his friend Eduardo Saverin with an idea for Thefacebook, a social networking website
    that would be exclusive to Ivy League students. Saverin provides $1,000 in seed funding, allowing Zuckerberg to
    build the website, which quickly becomes popular. When they learn of Thefacebook, the Winklevoss twins and
     Narendra are incensed, believing that Zuckerberg stole their idea while misleading them by stalling development
      on the Harvard Connection website. They raise their complaint with Harvard President Larry Summers,
      who is dismissive and sees no value in disciplinary action on Thefacebook or Zuckerberg.
"""

verbose_text = verbose_text.replace("\n","")

In [5]:
from transformers import pipeline


extractive_summarizer = pipeline("summarization",
                                 min_length=10,
                                 max_length=100)

#Extractive summarization
extractive_summary=extractive_summarizer(verbose_text)

print(extractive_summary[0].get("summary_text"))



 Mark Zuckerberg created a campus website called Facemash after he wrote an insulting post about his girlfriend . Zuckerberg is given six months of academic probation after the site crashes parts of Harvard's computer network . The Winklevoss twins and their business partner invite Zuckerberg to work on Harvard Connection, a social network .


In [7]:
print("Checkpoint used: ", extractive_summarizer.model.config)

Checkpoint used:  BartConfig {
  "_name_or_path": "sshleifer/distilbart-cnn-12-6",
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "extra_pos_embeddings": 2,
  "force_bos_token_to_be_generated": true,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_en

#Evaluating with ROUGE

In [9]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.8 MB/s eta 0:00:00


In [11]:
pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=870e36c322e331c7be9035adedba32eb45df768200d129d159c5caeb2dd3c511
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [12]:
import evaluate

rouge_evaluator = evaluate.load("rouge")

#Evaluate exact match strings
reference_text=["This is the same string"]
predict_text=["This is the same string"]

eval_results=rouge_evaluator.compute(predictions=predict_text,
                                     references=reference_text)
print("Results for Exact match",eval_results)


Results for Exact match {'rouge1': 1.0, 'rouge2': 1.0, 'rougeL': 1.0, 'rougeLsum': 1.0}


In [13]:
#Evaluate no-match strings
reference_text=["This is the different string"]
predict_text=["Google can predict warm weather"]

eval_results=rouge_evaluator.compute(predictions=predict_text,
                                     references=reference_text)
print("\nResults for no match", eval_results)


Results for no match {'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0, 'rougeLsum': 0.0}


In [14]:

#Evaluate summary
eval_results=rouge_evaluator.compute(
    predictions=[extractive_summary[0].get("summary_text")],
    references=[verbose_text])

print("\nResults for Summary generated", eval_results)


Results for Summary generated {'rouge1': 0.36641221374045796, 'rouge2': 0.29230769230769227, 'rougeL': 0.25190839694656486, 'rougeLsum': 0.25190839694656486}
